In [4]:
# Importamos las librerías necesarias
import datetime      # Para trabajar con las fechas (cada bloque tiene su propio timestamp)
import hashlib       # Para crear los hashes de los bloques, mediante el SHA256
import json          # Para crear ficheros .json, muchas funciones antes de "hashear" estarán en este formato
from flask import Flask, jsonify, request   # request = para conectarse a un nodo u otro de la cadena de bloques descentralizada
import requests
from uuid import uuid4    # Para establecer identificadores
from urllib.parse import urlparse   # Para poder parsear, conectar, recibir direcciones, etc... de la criptomoneda

## flask == 1.1.2
## requests == 2.25.1

# NOTA: No confundir la librería "request" del módulo flask con "requests"

---

Se va a utilizar TODO el código empleado en la cadena de bloques, pero con una tercera parte adicional:

  * **PARTE 1**: Crear la cadena de bloques
  * **PARTE 2**: Minar un bloque de la cadena de bloques
  * **PARTE 3**: Descentralizar la cadena de bloques
  
Aunque se va a emplear la misma estructura o "esqueleto" definida en la Blockchain, para la parte 1 y parte 2 se van a añadir fragmentos de código adicional para que el conjunto sea funcional. A este respecto, se han borrado todos los comentarios del anterior código y solo se han dejado aquellos comentarios donde se añade nuevo código.

---

## Parte 1 - Crear la cadena de bloques

Las diferencias entre una cadena de bloques "convencional" y una cadena de bloques empleada en criptomonedas es que esta última contiene:

 * **Transacciones** entre bloques.
 * Se establece un **protocolo de consenso** que hace que todos los nodos de la red descentralizada dispongan de la misma versión de la cadena.
 
<div class="alert alert-block alert-info">
<strong>Transacciones entre bloques</strong>
</div>
 
Cada vez que se mine un nuevo bloque, TODAS las transacciones que se hayan acumulado en la Mempool, **serán añadidas al bloque**. Esto es importante tenerlo en cuenta ya que las transacciones no formaban parte del bloque en el código original. Todas las transacciones se almacenarán en una lista, y cuando todas las transacciones se añadan al nuevo bloque, la lista se quedará vacía para poder reutilizara para almacenar nuevas transacciones. Así pues, <font color='red'>**la lista será una memoria temporal**</font>.

Es importante saber ubicar la creación de las transacciones dentro del código. Así pues, se hará dentro de `__init__` pero **antes** de la creación del bloque. En este caso, se hará entre:

  * `self.chain`
  * **`self.transactions`** <--------- AQUÍ IRÁN LAS TRANSACCIONES
  * `self.create_block(...)`

Cuando se crea el bloque, también habrá que añadir una quinta clave al diccionario `block` para que incluya las transacciones: `'transactions' : self.transactions`. Como una transacción concreta solo puede formar parte de un solo bloque, una vez que las transacciones han sido registradas en dicho bloque (es decir, justo después de minar un nuevo bloque) la lista tendrá que ser vaciada.

También habrá que añadir un nuevo método (**`def add_transaction`**) en la clase `Blockchain` que necesitará formatear una nueva transacción y añadirla a la lista de transacciones, que posteriormente será volcada en los nuevos bloques a medida que se vayan minando. Se añadirán tres variables:

  * `sender` = el que realiza el envío
  * `receiver` = el que recibe transacción
  * `amount` = la cantidad de criptomonedas que sea envían en la transacción
  
<div class="alert alert-block alert-info">
<strong>Protocolo de consenso</strong>
</div>
Antes que nada, habrá que inicializar un **conjunto** de nodos. En este caso, no se crea una lista, ya que la lista presupone un orden y los nodos pueden estar repartidos de forma aleatoria por todo el mundo, sin un orden concreto. Por eso se establece un conjunto `self.nodes = set()`, que vendría a ser una lista sin orden.

Ahora se creará otro método (**`def add_nodes`**) que tomará como argumentos `self` (para poder acceder a todas las variables de la clase `Blockchain`) y **la dirección del nuevo nodo** (**`address`**), en este caso, será la dirección empleada en Postman `http://127.0.0.1:5000`. En todos los nodos, la dirección URL seguirá siendo la misma (`http://127.0.0.1`), pero cada vez que se añada un nuevo nodo **cambiará el puerto** (`5000`, `5001`, `5002`...). NOTA: Esto es así porque se está usando una dirección propia, pero si los nodos estuvieran repartidos por todo el mundo, la URL también cambiaría según la ubicación del nodo.

El argumento `address` es para conocer la dirección donde se van a registrar los nuevos nodos. Mediante la función `urlparse` se procesarán las direcciones; lo que hace es "trocear" o dividir la dirección en los siguientes parámetros:

 * **`scheme`** = donde se guardará el protocolo (`http`, `https`, etc)
 * **`netloc`** = donde se guardarán los números de la dirección (`127.0.0.1:5000`)
 * **`path`** = el separador entre los componentes de la dirección (`/`)
 
Hay más parámetros, pero el que nos interesa en ese caso será el **`netloc`**, que será el campo que se escogerá de cada nodo mediante el código `self.nodes.add(parsed_url.netloc)`. NOTA: se utiliza `.add()` y no `.append()` porque, como ya se ha comentado anteriormente, los nodos son un **conjunto** y no una lista.

Luego se definirá un método **`replace_chain()`**, donde en cada nodo se sustituirá en el último eslabón aquel bloque con la cadena más larga. Para ello se declarará la variable `network`, que es el conjunto de TODOS los nodos del sistema, y la variable `longest_chain`, que será el nodo con la cadena más larga. Como al principio no se sabe quién tiene la cadena más larga, se inicializará con `longest_chain = None`. Se asumirá, por defecto, que nuestro nodo tiene la cadena más larga a no ser que encontremos otra cadena más larga en el sistema.

Para ello, se realizará una búsqueda por toda la red mediante un bucle `for()`. En la variable variable respuesta (`response`) hay que buscar la dirección, mediante la función `.get()` de la librería `requests`, de un nodo. La petición para la dirección de un nodo era `http://127.0.0.1:5000/get_chain`, pero no tiene sentido poner esto en el código ya que en esta dirección sólo estará un nodo de la cadena. Nótese que aquí, lo importante, es que **un nodo le hace una petición a otro nodo** (el usuario no hace una petición en Postman, por ejemplo).

Como cada nodo tendrá una dirección diferente, entonces hay que **reemplazar los números por el parámetro `netloc`,** y para ello se empleará el código `requests.get(f'http://{node}/get_chain')`. La variable auxiliar `node` es el fragmento que se está recorriendo en el bucle para buscar nodos en la red, y la `f` al principio es la sintaxis de Python para buscar cadenas de caracteres de forma explícita (por eso hay que poner `{node}` entre llaves).

Así pues, la respuesta `response` devolverá una tupla que consta de dos elementos: la respuesta en formato `.json` y el código de respuesta. Si todo ha ido bien, deberá devolver el código `200`; en cuyo caso se establecerá una condición: si el código devuelto es `200`, se devolverá la **longitud** de la cadena y la información de la propia cadena. Luego se añade una doble condición: 
  * 1) La cadena del otro tiene que tener mayor longitud que mi propia cadena
  * 2) La cadena del otro también tiene que ser válida 

Si se cumplen estas dos condiciones simultáneamente, entonces la cadena más larga será la del nodo encontrado (y no la nuestra). Si no se cumplen estas condiciones, la cadena más larga (`longest_chain`) seguirá siendo `None`. Si se cumplen las condiciones, el valor `None` deberá ser reemplazado por la cadena más larga que se haya encontrado a la hora de recorrer el bucle.

In [5]:
# PARTE 1: CREACIÓN DE LA CADENA DE BLOQUES

class Blockchain: 
    
    # Constructor de la cadena de bloques
    def __init__(self):
        self.chain = []     
        self.transactions = []    # Lista donde se almacenarán todas las transacciones
        self.create_block(proof = 1, previous_hash = '0')
        self.nodes = set()        # Se establece un conjunto de nodos (una lista sin orden concreto)
                                                   
    # Crear un nuevo bloque de la cadena  
    def create_block(self, proof, previous_hash):
        block = {'index' : len(self.chain)+1,  
                 'timestamp' : str(datetime.datetime.now()), 
                 'proof' : proof,          
                 'previous_hash' : previous_hash,
                 'transactions' : self.transactions}  # Se volcará la lista de transacciones que haya en ese momento
        self.transactions = []     # La lista de transacciones será vaciada una vez que se haya minado un nuevo bloque
        self.chain.append(block)
        return block 
    
    # Obtener el último bloque de la cadena
    def get_previous_block(self):
        return self.chain[-1] 
    
    # Prueba de Trabajo (PROOF OF WORK)
    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:5] == '00000':
                check_proof = True
            else:
                new_proof += 1 
        return new_proof
    
    # FUNCIÓN DE HASH
    def hash(self, block):  
        encoded_block = json.dumps(block, sort_keys=True).encode()  
        return hashlib.sha256(encoded_block).hexdigest()
    
    # COMPROBAR SI LA CADENA DE BLOQUES ES VÁLIDA
    def is_chain_valid(self, chain):
        previous_block = chain[0]  
        block_index = 1 
        while block_index < len(chain):  
            block = chain[block_index]  
            if block['previous_hash'] != self.HASH(previous_block):
                return False                           
            
            previous_proof = previous_block['proof']  
            proof = block['proof']                    
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode())  
            
            if hash_operation[:5] != '00000':
                return False    
            previous_block = block   
            block_index += 1         
        return True
    
    # AÑADIR TRANSACCIONES
    def add_transaction(self, sender, receiver, amount):  # 3 variables: el que envía, el que recibe y la cantidad 
        # Se añade una transacción a la lista
        self.transactions.append({'sender': sender,
                                  'receiver' : receiver,
                                  'amount' : amount})
        previous_block = self.previous_block()  # Con esto se obtiene el último bloque minado...
        return previous_block['index'] + 1      # ...y se obtiene cuál será el siguiente bloque de la cadena
    
    # AÑADIR NODOS
    def add_nodes(self, address):
        parsed_url = urlparse(address)      # Se "parsea" (o trocea) la URL en los diferentes parámetros de la dirección
        self.nodes.add(parsed_url.netloc)   # Nos quedamos con el único parámetro que nos interesa: el "netloc"
    
    # REEMPLAZAR LA CADENA MÁS LARGA
    def replace_chain(self):
        network = self.nodes   # El conjunto de TODOS los nodos se llamará "network"
        longest_chain = None   # Se declara una variable con la cadena más larga (todavía no se sabe a quién corresponde)
        max_length = len(self.chain)   # La longitud más larga es nuestra cadena (a menos que exista otra más larga en la red)
        for node in network:
            response = requests.get(f'http://{node}/get_chain')  # Se busca el "netloc" en la URL de cada nodo
            if response.status_code == 200:
                length = response.json()['length']   # Del diccionario .json, extraemos el parámetro 'length'
                chain = response.json()['chain']     # Del diccionario .json, extraemos la cadena 'chain'
                if length > max_length and self.is_chain_valid(chain):  # Si la cadena del otro es más larga y es válida...
                    max_length = length        # ... la longitud máxima no es la nuestra, sino la del nodo recorrido (length)
                    longest_chain = chain      # ... la cadena más larga ya no es None, si no la que se ha encontrado (chain)
        if longest_chain:
            self.chain = longest_chain    # Se reemplaza el valor None por el valor de la cadena más larga
            return True
        return False

---

## Parte 2 - Minado de una cadena de bloques

<div class="alert alert-block alert-info">
<strong>Generar una dirección para el nodo</strong>
</div>

Conviene recordar que, cuando se definió la cadena de bloques, se creó una aplicación web basada en Flask para poder interactuar con la blockchain. Así pues, ahora se indicará la **dirección exacta** y **el puerto del nodo** que va a ejecutar (lanzar) la cadena de bloques. El puerto era el 5000, pero en un momento dado, cada nodo tendría un puerto diferente (5001, 5002, 5003...)

¿Cómo se puede crear una dirección para el nodo que se ubicará en el puerto 5000? Lo primero que hay que tener en cuenta es que, cuando se mina un nuevo bloque, ocurre una transacción a favor del minero que obtiene la recompensa en criptomonedas. Por eso se necesita la dirección del nodo, porque es la que se utilizará como **emisor** de la criptomoneda cuando el minero mina un nuevo bloque. El emisor de la transacción será el nodo donde se ha registrado el nuevo bloque de la cadena y la dirección de recepción (es decir, el que recibirá la criptomoneda será el propio minero). La **primera transacción de cualquier bloque** ocurre cuando el minero mina un nuevo bloque: es justo en ese momento cuando el minero recibe la recompensa oculta detrás del minado de ese bloque. 

Así que, volviendo a la pregunta: ¿cómo se puede crear esa dirección? Se empleará la función `uuid4` de la librería `uuid` (*Universally Unique IDentifier*), para **generar un identificador único aleatorio**. El identificador UUID está formado por 32 dígitos o caracteres divididos en 5 grupos, y cada grupo separado por guiones; este sería el formato:

$$8-4-4-4-12$$

Para este propósito, se dejarán únicamente los números, de manera que se eliminarán los guiones. De manera que el resultado se convertirá en una cadena (*string*) para poder quitarle los guiones: `node_address = str(uuid4()).replace('-', '')` (esta línea de código se añadirá antes de la creación de la *blockchain*).

<div class="alert alert-block alert-info">
<strong>Minar un nuevo bloque de la criptomoneda</strong>
</div>

El siguiente paso será ajustar la petición de minar un bloque (`'/mine_block'`) para vincularla a la función homónima (`def mine_block`), y esto servirá para **convertir la cadena de bloques en una criptomoneda**. Para que sea una criptomoneda y no simplemente una cadena de bloques con información genérica, hay que **incluir las transacciones**. Para ello, hay que hacer un par de modificaciones al código:

   * Incluir una nueva clave llamada `transactions` al final del diccionario `response`, para incluirla como variable del propio bloque.
   * Antes de crear el propio bloque como tal, hay que incluir una transacción del propio minero: la que obtiene como recompensa por minar un nuevo bloque. Hay que tener cuidado donde incluir esta transacción, y su lugar será entre el hash previo (`previous_hash`) y el bloque (`block`). ¿Por qué ahí? Porque la última transacción que hay que añadir es, precisamente, la del propio minero que obtiene como recompensa. Como ya se tienen los "ingredientes" para minar un bloque mediante la función **`add_transaction`**, solo hay que llamar a dicha función para añadir esa transacción adicional por haber minado un nuevo bloque. Para ello hay que definir un emisor, un receptor y una cantidad de recompensa:
   
      * `sender` = el que emite es el "productor", en este caso será el nodo `new_address`.
      * `receiver` = la persona que va a recibir la recompensa. En este caso se pondrá "Alberto", porque en esta demostración será una sola persona quien vaya a minar los bloques.
      * `amount` = la cantidad de recompensa en criptomonedas que se recibirán por minar un nuevo bloque. Hay que tener en cuenta que encontrar la prueba de trabajo (*Proof of Work*) con sólo 5 ceros delante es relativamente sencillo, así que hay que definir una recompensa ajustada a la dificultad del puzzle criptográfico. En este caso, pues se han puesto 5 criptomonedas.
      
De esta manera, la última transacción es la del propio minero al minar un nuevo bloque, y luego se crea el bloque con todas las transacciones (incluida la del minero), y tras devolver ese bloque creado por la cadena de bloques, la variable `block` contendrá la nueva variable `transactions` (que se devuelve al final en la `response` jsonificada.

<div class="alert alert-block alert-info">
<strong>Añadir una nueva transacción por POST</strong>
</div>

A continuación se definirán peticiones a través de **POST** para poder añadir una nueva transacción al *pool* de transacciones. Lo primero que hay que diferenciar es que, en una llamada con `GET` (como se había hecho en la *blockchain*), se obtiene una respuesta que contiene, por ejemplo, el nuevo bloque que ha sido minado, sin necesidad de crear absolutamente nada para obtener ese nuevo bloque. Sin embargo, en una llamada tipo **`POST`** hay que **crear algo** para obtener esa respuesta (de ahí el nombre `POST`, porque se está "posteando" algo dentro del cliente `http`).

Para la nueva petición `POST`, el nuevo punto de ruta será `/add_transaction`. Luego se definirá una función que se llamará igual que el *web service*: `add_transaction` (NOTA: <font color='red'>**no es necesario que se llamen igual**</font>, se hace así simplemente para dar consistencia al código). Como el fichero JSON que contiene toda la información (`sender`, `receiver` y `amount`) tiene que ser "posteado" en Postman, mediante la librería `request` se obtendrá el fichero JSON posteado (enviado por `POST`) a través de Postman.

Para asegurarse que no falta ninguna de las claves de la transacción (emisor, receptor y cantidad), se definirá una variable `transaction_keys` que contendrá las tres claves, y luego se definirá una varible `index` para obtener el índice del siguiente bloque que contendrá esa transacción en concreto con las tres claves. Finalmente, si se hace correctamente la petición, se devolverá la respuesta **201**, que es la respuesta *web service* cuando se crea algo correctamente a través de una petición `POST`.

In [8]:
# PARTE 2 - MINADO DE UN BlOQUE DE LA CADENA

#  Crear una aplicación web (web app)
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Crear la dirección del nodo en el puerto 5000
node_address = str(uuid4()).replace('-', '')  # Se reemplazan los guiones por un carácter vacío

#  Crear una blockchain
blockchain = Blockchain()

#### ------------------------------------------------------------------------

# PRIMERA PETICIÓN: minado de un nuevo bloque
@app.route('/mine_block', methods = ['GET'])

def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    blockchain.add_transaction(sender = node_address,      # Se añade la última transacción de recompensa a la cadena
                               receiver = "Alberto",
                               amount = 5)
    block = blockchain.create_block(proof, previous_hash)
    # Una vez creado el bloque, añadimos una respuesta:
    response = {'message' : 'Se ha minado un nuevo bloque',
                'index' : block['index'],
                'timestamp' : block['timestamp'],
                'proof' : block['proof'],
                'previous_hash' : block['previous_hash'],
                'transactions' : block['transactions']}    # Se añaden las transacciones para definir la criptomoneda
    return jsonify(response), 200

#### ------------------------------------------------------------------------

# SEGUNDA PETICIÓN: obtener la cadena de bloques entera
@app.route('/get_chain', methods = ['GET'])

def get_chain():  
    response = {'chain' : blockchain.chain,
                'longitud cadena' : len(blockchain.chain)}
    
    return jsonify(response), 200


#### ------------------------------------------------------------------------


# Añadir una nueva transacción a la cadena de bloques
@app.route('/add_transaction', methods = ['POST'])

def add_transaction(): 
    json = request.get_json()  # La librería "request" es capaz de obtener el fichero JSON enviado por POST a través de Postman
    transaction_keys = ['sender', 'receiver', 'amount']   # Definimos las tres claves de una transacción
    if not all(key in json for key in transaction_keys):   # Si no están las tres claves de una transacción dará error
        return 'Faltan elementos de la transacción', 400    # El código 400 corresponde a errores del cliente
    index = blockchain.add_transaction(json['sender'], json['receiver'], json['amount'])
    response = {'message' : f'La transacción será añadida al bloque número {index}'}
    return jsonify(response), 201


NOTA: si se obtiene un error `500 Internal Server Error`, hay que actualizar la librería Flask, reiniciar Spyder/Jupyter/etc y ejecutar la siguiente línea de código (se puede incluir con el resto del código):

`app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False`

---

## Parte 3 - Descentralizar la cadena de bloques

Comos se ha comentado al inicio, la diferencia entre una cadena de bloques "convencional" y una cadena de bloques empleada en criptomonedas es que esta última contiene **transacciones**. Hasta ahora, las transacciones las ha generado un único nodo, pero... ¿Qué pasa si se quieren añadir más nodos a la red? Si se quieren añadir (dar de alta) nuevos nodos, la red donde suceden todas estas transacciones deberá estar **descentralizada** y para ello hacen falta dos peticiones adicionales: conectar un nuevo nodo a la red descentralizada y reemplazar la cadena de cualquier nodo que no esté actualizado.

<div class="alert alert-block alert-info">
<strong>Petición para dar de alta nuevos nodos en la red</strong>
</div>

A través de `POST` de Postman, se va a enviar un fichero JSON que contenga los nuevos nodos a dar de alta. El punto de ruta esta ver será `/connect_node` y la función a definir será `connect_node` (se recuerda que se hace así para dar consistencia al código, pero no es obligatorio que se llamen igual). La función no tomará ningún argumento porque las variables se tomarán de la cadena de bloques, y la variable `json` a partir de la cual se obtendrá toda la información se puede llamar igual que en anteriores peticiones, ya que están separadas y las peticiones son independientes.

Ahora hay que **conectar un nuevo nodo a los nodos que ya existen en la red**. Primero se necesita la dirección del nodo a dar de alta, y para ello hay que procesar el JSON para extraer el valor del nodo (o los nodos) que hay que dar de alta, y para ello se añade una variable `nodes` que contengan (a través del JSON) una lista con todos los nodos nuevos. Luego se hace una comprobación para ver si esta lista no esté vacía y efectivamente contenga algún nodo (para evitar posibles errores).

Mediante un bucle `for` se recorren todos los nodos que hay que dar de alta en la lista; de este modo, `node` será el parámetro de entrada del método `add_node` del objeto `blockchain`. Así pues, se recorre el bucle para que la cadena de bloques añada un nuevo nodo pasándole como dirección el valor de `node`. Una vez recorrido el bucle y se hayan añadido todos los nuevos nodos, se devolverá una respuesta afirmativa. 

<div class="alert alert-block alert-info">
<strong>Petición para reemplazar la cadena de bloques más larga</strong>
</div>

En este caso, la petición se realizará mediante `GET` y no por `POST`, ya que no hay nada que crear. La petición consistirá en verificar si todos y cada uno de los nodos de la red tienen la cadena más larga o no. 

Mediante la variable de verificación `is_chain_replaced`, se establecerá una comprobación *booleana* (verdadero o falso) en función de si se necesita o no reemplazar la cadena de bloques. 

Se han denominado dos variables: 

  * `new_chain`: para hacer énfasis que la nueva cadena es diferente a la original
  * `actual_chain`: para indicar que no ha cambiado absolutamente nada y la cadena sigue siendo la misma

In [9]:
# PARTE 3 - DESCENTRALIZAR LA CADENA DE BLOQUES


# PRIMERA PETICIÓN: Conectar nuevos nodos

@app.route('/connect_node', methods = ['POST'])
def connect_node():
    json = request.get_json()   # La variable se puede llamar "json" igual que en las peticiones anteriores
    nodes = json.get('nodes')   # Esta variable contiene los nodos nuevos a añadir a la red
    if nodes is None:
        return "No hay nodos nuevos para añadir", 400   # Devuelve error de web service 400 si no hay ningún nodo
    for node in nodes:      # Para cada uno de los nodos de la lista de nodos...
        blockchain.add_node(node)   # ...la cadena de bloques añade un nuevo nodo pasándole como dirección el valor de "node"
    response = {'message' : 'Se han conectado todos los nodos. La cadena de criptomonedas contiene los nodos siguientes: ',
                'total' : list(blockchain.nodes)}   # Se devuelve una lista de todos los nodos de forma más "visual"
    return jsonify(response), 201


# SEGUNDA PETICIÓN: Reemplazar la cadena por la más larga (si es necesario)

@app.route('/replace_chain', methods = ['GET'])
def replace_chain():
    is_chain_replaced = blockchain.replace_chain()    # is_chain_replaced será un valor booleano (verdadero o falso)
    if is_chain_replaced:
        response = {'message' : 'Los nodos tenían diferentes cadenas que han sido todas reemplazadas por la más larga.',
                    'new_chain' : blockchain.chain}  # Devuelve la nueva cadena (se ha reemplazado)
    else:
        response = {'message' : 'Todo correcto: la cadena ya es la más larga.',
                    'actual_chain' : blockchain.chain}   # Devuelve la cadena actual (no ha cambiado nada)
    return jsonify(response), 200

---

## Ejecutar la app

Antes de ejecutar la app, hay que tener preparados dos archivos JSON: uno con las direcciones de los nodos (5001, 5002, 5003...) y otro JSON con las 3 claves para realizar una transacción: `sender`, `receiver` y `amount`. Luego ya se podrá ejecutar la app y realizar peticiones en Postman.

In [ ]:
# Hay que ejecutar la app para los 3 nodos de prueba
app.run(host = '0.0.0.0', port = 5000)
app.run(host = '0.0.0.0', port = 5001)
app.run(host = '0.0.0.0', port = 5002)
app.run(host = '0.0.0.0', port = 5003)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.42.233:5000/ (Press CTRL+C to quit)
192.168.42.233 - - [11/Jul/2022 01:54:25] "GET /get_chain HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:54:32] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:54:36] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:54:40] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:54:42] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:54:54] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:54:58] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:55:03] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:55:09] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:55:12] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:55:21] "GET /mine_block HTTP/1.1" 200 -
192.168.42.233 - - [11/Jul/2022 01:55:25] "GET /mine_block HTTP/1.1" 200 -
192.168